## 공급망 데이터 분석
목표: 기간별 각 Source 기업의 Suppliers들의 ESG 등급별 갯수

1. SC 샘플 데이터 및 Supplier들의 ESG 데이터 개별 호출
2. 합병
3. 피벗

* 분석에 필요한 칼럼/데이터만 축약해서 리서치를 진행하는 것이 중요!

In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('supply_chain_sample.xlsx')

In [8]:
# 시간까지 포함한 날짜 데이터를 str으로 변환해서 첫 10글자만 추출 후, datetime으로 변경

df['target_start_date'] = pd.to_datetime(df['target_start_date'].astype(str).apply(lambda x: x[:10]))

In [10]:
# tab delimited txt 호출 방법

df2 = pd.read_csv('SC links company data.txt', sep='\t')

In [13]:
df.columns

Index(['source_factset_entity_id', 'source_entity_name', 'relationship_type',
       'source_fsym_id', 'source_ticker_region', 'source_isin', 'source_cusip',
       'source_sedol', 'target_start_date', 'target_end_date',
       'target_factset_entity_id', 'target_entity_name', 'target_revenue_pct',
       'target_primary_ticker', 'target_country', 'target_entity_type',
       'target_fsym_id', 'target_ticker_region', 'target_isin', 'target_cusip',
       'target_sedol'],
      dtype='object')

In [14]:
df

,source_factset_entity_id,source_entity_name,relationship_type,source_fsym_id,source_ticker_region,source_isin,source_cusip,source_sedol,target_start_date,target_end_date,...,target_entity_name,target_revenue_pct,target_primary_ticker,target_country,target_entity_type,target_fsym_id,target_ticker_region,target_isin,target_cusip,target_sedol
0,000C7F-E,"Apple, Inc.",Partner - Manufacturing,R85KLC-S,AAPL-US,US0378331005,37833100,2046251,2018-11-22,NaN,...,Finisar Corp.,NaN,FNSR-US,United States,Public Company,DVQ55Z-S,FNSR-US,US31787A5074,31787A507,2517832
1,000C7F-E,"Apple, Inc.",Partner - Research Collaboration,R85KLC-S,AAPL-US,US0378331005,37833100,2046251,2018-11-22,NaN,...,Aon Plc,NaN,AON-US,United Kingdom,Public Company,H9T6DW-S,AON-US,GB00B5BT0K07,G0408V102,B5BT0K0
2,000C7F-E,"Apple, Inc.",Partner - Research Collaboration,R85KLC-S,AAPL-US,US0378331005,37833100,2046251,2018-11-22,NaN,...,General Electric Co.,NaN,GE-US,United States,Public Company,P321KV-S,GE-US,US3696041033,369604103,2380498
3,000C7F-E,"Apple, Inc.",Supplier,R85KLC-S,AAPL-US,US0378331005,37833100,2046251,2012-01-17,NaN,...,Amphenol Corp.,NaN,APH-US,United States,Public Company,WL12NL-S,APH-US,US0320951017,32095101,2145084
4,000C7F-E,"Apple, Inc.",Supplier,R85KLC-S,AAPL-US,US0378331005,37833100,2046251,2012-01-17,NaN,...,"Maxim Integrated Products, Inc.",NaN,MXIM-US,United States,Public Company,V0BDBS-S,MXIM-US,US57772K1016,57772K101,2573760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,0FPWZZ-E,"Alphabet, Inc.",Customer,XF9TK6-S,GOOGL-US,US02079K3059,02079K305,BYVY8G0,2019-02-12,NaN,...,Air France-KLM SA,NaN,AF-FR,France,Public Company,WN9127-S,AF-FR,FR0000031122,F01699135,4916039
528,0FPWZZ-E,"Alphabet, Inc.",Customer,XF9TK6-S,GOOGL-US,US02079K3059,02079K305,BYVY8G0,2019-02-12,NaN,...,"The Motley Fool Holdings, Inc.",NaN,NaN,United States,Holding Company,NaN,NaN,NaN,NaN,NaN
529,0FPWZZ-E,"Alphabet, Inc.",Customer,XF9TK6-S,GOOGL-US,US02079K3059,02079K305,BYVY8G0,2019-02-12,NaN,...,BookIt Operating LLC,NaN,NaN,United States,Private Company,NaN,NaN,NaN,NaN,NaN
530,0FPWZZ-E,"Alphabet, Inc.",Partner - Research Collaboration,XF9TK6-S,GOOGL-US,US02079K3059,02079K305,BYVY8G0,2019-02-11,NaN,...,SAP SE,NaN,SAP-DE,Germany,Public Company,F3JG9C-S,SAP-DE,DE0007164600,D66992104,4846288


In [15]:
df1 = df[['source_entity_name', 'relationship_type', 'source_ticker_region', 'target_start_date', 'target_entity_name', 'target_primary_ticker', 'target_country']]

In [16]:
df1

,source_entity_name,relationship_type,source_ticker_region,target_start_date,target_entity_name,target_primary_ticker,target_country
0,"Apple, Inc.",Partner - Manufacturing,AAPL-US,2018-11-22,Finisar Corp.,FNSR-US,United States
1,"Apple, Inc.",Partner - Research Collaboration,AAPL-US,2018-11-22,Aon Plc,AON-US,United Kingdom
2,"Apple, Inc.",Partner - Research Collaboration,AAPL-US,2018-11-22,General Electric Co.,GE-US,United States
3,"Apple, Inc.",Supplier,AAPL-US,2012-01-17,Amphenol Corp.,APH-US,United States
4,"Apple, Inc.",Supplier,AAPL-US,2012-01-17,"Maxim Integrated Products, Inc.",MXIM-US,United States
...,...,...,...,...,...,...,...
527,"Alphabet, Inc.",Customer,GOOGL-US,2019-02-12,Air France-KLM SA,AF-FR,France
528,"Alphabet, Inc.",Customer,GOOGL-US,2019-02-12,"The Motley Fool Holdings, Inc.",NaN,United States
529,"Alphabet, Inc.",Customer,GOOGL-US,2019-02-12,BookIt Operating LLC,NaN,United States
530,"Alphabet, Inc.",Partner - Research Collaboration,GOOGL-US,2019-02-11,SAP SE,SAP-DE,Germany


In [17]:
df2['Periods'] = pd.to_datetime(df2['Periods'])

In [77]:
df2

,Unnamed: 0,Company Name,Periods,Ticker,Universe Returns,Country,LT Debt to Capital,LT Debt to Equity,Cash Ratio,Debt Service Ratio,Beneish M-Score,Net Income Stability,ESG Rating,Credit Rating,Credit Rating - Numeric
0,00287Y10,"AbbVie, Inc.",2014-12-31,ABBV-US,-7.03,US,0.72,2.77,1.62,5.81,-2.71,5.68,BB,A,700.0
1,00287Y10,"AbbVie, Inc.",2015-01-30,ABBV-US,0.25,US,0.72,2.77,1.62,5.81,-2.71,5.68,BB,A,700.0
2,00287Y10,"AbbVie, Inc.",2015-02-27,ABBV-US,-3.24,US,0.63,6.06,0.73,1.24,-2.86,4.15,BB,A,700.0
3,00287Y10,"AbbVie, Inc.",2015-03-31,ABBV-US,11.33,US,0.72,2.77,1.62,5.81,-2.71,5.68,BB,A,700.0
4,00287Y10,"AbbVie, Inc.",2015-04-30,ABBV-US,2.98,US,0.74,3.11,1.56,5.81,-2.84,5.64,BB,A,700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28130,Y2573F10,Flex Ltd.,2021-08-31,FLEX-US,-4.84,SG,0.51,1.16,0.33,1.64,-2.41,1.35,AA,BBB-,660.0
28131,Y2573F10,Flex Ltd.,2021-09-30,FLEX-US,-4.41,SG,0.51,1.16,0.33,1.64,-2.41,1.35,AA,BBB-,660.0
28132,Y2573F10,Flex Ltd.,2021-10-29,FLEX-US,1.18,SG,0.51,1.16,0.33,1.64,-2.41,1.35,AA,BBB-,660.0
28133,Y2573F10,Flex Ltd.,2021-11-30,FLEX-US,7.19,SG,0.52,1.15,0.28,1.64,-2.31,1.14,AA,BBB-,660.0


In [19]:
df2_filtered = df2[['Company Name', 'Periods', 'Ticker', 'Beneish M-Score', 'ESG Rating']]

In [20]:
df2_filtered

,Company Name,Periods,Ticker,Beneish M-Score,ESG Rating
0,"AbbVie, Inc.",2014-12-31,ABBV-US,-2.71,BB
1,"AbbVie, Inc.",2015-01-30,ABBV-US,-2.71,BB
2,"AbbVie, Inc.",2015-02-27,ABBV-US,-2.86,BB
3,"AbbVie, Inc.",2015-03-31,ABBV-US,-2.71,BB
4,"AbbVie, Inc.",2015-04-30,ABBV-US,-2.84,BB
...,...,...,...,...,...
28130,Flex Ltd.,2021-08-31,FLEX-US,-2.41,AA
28131,Flex Ltd.,2021-09-30,FLEX-US,-2.41,AA
28132,Flex Ltd.,2021-10-29,FLEX-US,-2.41,AA
28133,Flex Ltd.,2021-11-30,FLEX-US,-2.31,AA


In [32]:
# filtering 할 때, 등호(==)를 사용할 수도 있지만, isin을 사용하면 list를 손쉽게 사용할 수 있음 (or 로직 적용).

df1_filtered = df1.dropna(subset=['target_primary_ticker'])
df1_filtered = df1_filtered[df1_filtered['relationship_type'].isin(['Supplier'])]

In [33]:
df1_filtered = df1_filtered.sort_values(by=['target_start_date'])
df2_filtered = df2_filtered.sort_values(by=['Periods'])

In [34]:
# direction = backward, forward, nearest
# 가장 가까운 과거 데이터를 찾을때는 backward 사용.

merged_df = pd.merge_asof(df1_filtered, df2_filtered, left_on=['target_start_date'], right_on=['Periods'], direction='backward', left_by='target_primary_ticker', right_by='Ticker')

In [35]:
merged_df = merged_df.dropna(subset=['ESG Rating'])

In [69]:
# aggfunc = len 갯수

pivot_table = pd.pivot_table(merged_df[['source_entity_name', 'target_start_date', 'ESG Rating']], index=['source_entity_name', 'target_start_date'], columns='ESG Rating', aggfunc=len).fillna(0)

In [70]:
pivot_table = pivot_table[['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC']]

In [71]:
# 우측 끝에 합 칼럼 추가

pivot_table['total'] = pivot_table[['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC']].sum(axis=1)

In [73]:
# applymap(int)로 소수점 제거

pivot_table = pivot_table.applymap(int)

In [75]:
# 0은 공란으로 표시하도록

pivot_table = pivot_table.applymap(lambda a: '' if a == 0 else a)

In [76]:
pivot_table

ESG Rating                           AAA AA  A BBB BB  B CCC  total
source_entity_name target_start_date                               
Apple, Inc.        2015-11-03                2   1     1          4
                   2015-11-04                       1  1          2
                   2016-02-09                       1             1
                   2016-03-31                    1     1          2
                   2016-04-01             1  1   3  1             6
                   2017-11-07                2                    2
                   2017-11-16             2      2  2             6
                   2017-12-12                          1          1
                   2017-12-13                    1                1
                   2018-11-22          1  1  2   2     7   1     14
                   2019-02-08             1                       1
BP Plc             2015-03-05                1                    1
                   2018-05-03             1                       1